# Computation time comparison fooof vs IRASA
Tested on September 22nd 2021

In [4]:
import mne
import numpy as np
import scipy.signal as sig
from fooof import FOOOFGroup

from utils import irasa

#### Load data

In [5]:
# File names
path = "../data/Fig5/"
fname = "subj6_off_R1_raw.fif"
sub = mne.io.read_raw_fif(path + fname, preload=True)

# Convert mne to numpy
sample_rate = 2400
start = int(0.5 * sample_rate)  # artefact in beginning of recording
stop = int(185 * sample_rate)  # artefact at the end of recording

sub = sub.get_data(start=start, stop=stop)[:9]
band = (1, 30)
irasa_params = dict(data=sub, sf=sample_rate, band=band)

welch_params_fooof = dict(fs=sample_rate, nperseg=sample_rate)
welch_params_fooof_highRes = dict(fs=sample_rate, nperseg=4*sample_rate)

Opening raw data file ../data/Fig5/subj6_off_R1_raw.fif...
Isotrak not found
    Range : 0 ... 463199 =      0.000 ...   193.000 secs
Ready.
Reading 0 ... 463199  =      0.000 ...   193.000 secs...


#### IRASA at default parameters: 13.0 +- 0.3 seconds

In [6]:
%timeit IRASA = irasa(**irasa_params)

13.4 s ± 166 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### IRASA does not slow down if spectral resolution is decreased: 13.1 +- 0.3 seconds

In [8]:
%timeit IRASA = irasa(win_sec=20, **irasa_params)

13.1 s ± 297 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### IRASA does not slow down if spectral resolution is increased: 12.3 +- 0.6 seconds

In [9]:
%timeit IRASA = irasa(win_sec=0.5, **irasa_params)

12.3 s ± 606 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### IRASA slows down if number of hset is increased: 59.4 +- 1.2 seconds

In [10]:
%timeit IRASA = irasa(hset=np.arange(1.1, 1.9, 0.01), **irasa_params)

59.4 s ± 1.19 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### IRASA slows down if values of hset is increased: 64 +- 2 seconds

In [15]:
%timeit IRASA = irasa(hset=np.arange(10.1, 10.9, 0.05), **irasa_params)

1min 4s ± 1.83 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Fooof is fast even if psd calculation is included: 269 +- 2 miliseconds

In [24]:
%%timeit
freq, psd_sub = sig.welch(sub, **welch_params_fooof)

fm = FOOOFGroup(verbose=False)
fm.fit(freq, psd_sub, band)

278 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Fooof is twice as fast without PSD calculcation: 126 +- 6 miliseconds

In [21]:
freq, psd_sub = sig.welch(sub, **welch_params_fooof)

In [23]:
%%timeit
fm = FOOOFGroup(verbose=False)
fm.fit(freq, psd_sub, band)

126 ms ± 5.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Fooof is slow if the spectral resolution is high: 418 +- 3 miliseconds

In [25]:
%%timeit

freq, psd_sub = sig.welch(sub, **welch_params_fooof_highRes)

fm = FOOOFGroup(verbose=False)
fm.fit(freq, psd_sub, band)

418 ms ± 3.49 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
